In [1]:
# Importaciones de librerias y src
import sys
sys.path.append("../")
from src import llamar_api_demanda as demanda
from src import llamar_api_generacion as generacion

In [2]:
demanda.guardar_datos_en_csv()

In [3]:
generacion.guardar_datos_en_csv()